# LSTMハンズオン

はじめに、機械学習とニューラルネットワークについて少し説明します。

機械学習はトレーニングデータと答えさえあればロジックを考えなくても最適な答えを出してくれる素敵な数学モデルです。(教師なし学習など、一部例外はあります)

ニューラルネットワークは機械学習の一種で、wikiから抜粋すると、脳機能に見られるいくつかの特性を計算機上のシミュレーションによって表現することを目指した数学モデルです。層を用いた学習方法が特徴で、層を深くした(その分計算コストをかなり増える)モデルをディープニューラルネットワークと言います。

<img src="volume/images/ml_venn.png" width="400">

RNNはニューラルネットワークの一種で、文章など連続的な情報を利用できる、時系列データに威力を発揮するモデルです。

株価などの数値データも時系列データですし、文章も単語の流れとして捉えて処理することができます。そして以前に計算された情報を覚えるための記憶力を持っていることが特徴です。

LSTMはRNNの一種で、RNNを改良した上位互換のモデルで、下記のようなタスクをこなせます。(できることの一部です)

- 連続した数値を入力して、数値を出力
- 画像を入力としてそのキャプション(文字)を出力
- 音声を入力として文字を出力

[参考](https://deepage.net/deep_learning/2017/05/23/recurrent-neural-networks.html)

今回はkeras(ニューラルネットワークのライブラリ)を利用した時系列データの予測を行います。

- 規則性のある波を予測 (値1つで予測)
- 規則性のあるアイスクリームの売り上げ予測 (売り上げだけではなく気温など複数の値を考慮して予測)
- 規則性のないbitcoinの値段を予測 (最終取引価格や最高売り価格などで予測)
- おまけでword2vec

In [ ]:
# jupiter上でグラフ表示
import matplotlib.pyplot as plt
%matplotlib inline

## 1 規則性のある波を予測 (値1つで予測)

### 1-1 データセットの作成

学習する上でまずデータセットが必要になります。データセットは、入力と出力、今回は時系列データとその後の値が必要になります。

まずは学習に使うデータを作ってみましょう。

In [ ]:
import numpy as np

In [ ]:
# sin関数で波生成
def sin(length=100, curve=4):
    x = np.arange(0, length)
    return np.sin((curve * np.pi) * x / length)

In [ ]:
plt.plot(sin())
plt.show()

このままでは単純にsinを計算すれば求められるので、ある程度ノイズを乗せて予測しずらそうなデータにしてみましょう。

In [ ]:
# ノイズ付きsin波
def noised_sin(length=100, curve=4, noise_rate=0.3, low=-1.0, high=1.0):
    x = sin(length=length, curve=curve)
    noise = noise_rate * np.random.uniform(low=low, high=high, size=len(x))
    return x + noise

In [ ]:
plt.plot(noised_sin())
plt.show()

ノイズを乗せれたので今度はデータを時系列データとして整形します。

時系列データの学習では数個の連続したデータを1セットとして用意する必要があります。

以下の関数で配列をstep個に分けたデータセットと答え(次の値)を作れます。

戻り値はnumpyで作られたarray like object(配列っぽいオブジェクト)です。

In [ ]:
def create_reccurent_dataset(vector_data, step=25):
    data, target = [], []
    for i in range(len(vector_data) - step):
        data.append(vector_data[i:(i + step)])
        target.append(vector_data[i + step])
    reshape_data = np.array(data).reshape(len(data), step, 1)
    reshape_label = np.array(target).reshape(len(data), 1)
    return reshape_data, reshape_label

適当な値を入れてみて、データの形状と中身の確認をしてみましょう。

shapeプロパティで形状を確認できます。多次元配列版 lengthのようなものです。

In [ ]:
train_data, train_label = create_reccurent_dataset([1,2,3,4,5,6,7,8,9], 3)

In [ ]:
train_data.shape

3個の連続した値を6つ持った配列になりました。

具体的にトレーニングデータは、(学習データ総数, 1学習の時系列数(ステップ数), 特徴量)で整形する必要があります。

3次元配列で出てきますが、他にも気温、湿度、曜日など予測に役立ちそうなデータを入れられます。(今回は1つのデータなので三番目は1になってます)

次に中身も確認してみましょう。

In [ ]:
train_data

連続した値を3つずつ分割して計6個の学習用データが入っています。

次に答えラベル(次に来るべき値)も確認してみます。

In [ ]:
train_label.shape

トレーニングデータ6個に対して6つの答えラベルが用意できています。

中身も確認してみましょう

In [ ]:
train_label

[1,2,3]の答えとして4、[2,3,4]の答えとして5...というようにトレーニングデータに対しての答えも用意できました。

では、実際にノイズつきデータをデータセットとして整形してみましょう。

length(データ個数)やnoise_rate(ノイズの度合い)をあげたり、curve(カーブ数)を変えてもらっても構いません！

In [ ]:
# ノイズ付きsin波予測
data = noised_sin(length=100, curve=4, noise_rate=0.3)
train_data, train_label = create_reccurent_dataset(data, step=25)

In [ ]:
train_data.shape

In [ ]:
train_label.shape

パラメータ変えていなければ25個の連続したデータが75個できたと思います。

### 1-2 モデル構築

時系列(長短期記憶)学習モデルは今回使用するLSTM以外にもたくさんあります。QRNNが新しく、良さそう。

今回はLSTMを使います。

- RNN
- LSTM
- [QRNN](https://qiita.com/icoxfog417/items/d77912e10a7c60ae680e)

tensorflow内蔵のkerasを利用します。(kerasはtensorflowのラッパーです)

In [ ]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Activation, LSTM
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.callbacks import EarlyStopping

step_count = 1つの学習データのStep数(今回は25)

feature_count = 特徴量 今回の学習するデータは1個のみ

hidden_unit_count = 中間層の数 ([公式](https://keras.io/ja/layers/recurrent/#lstm)の説明は出力の次元数ですが、LSTMの次に全層結合を挟むので中間層の数になります)

In [ ]:
step_count = 25
feature_count = 1
hidden_unit_count = 300

In [ ]:
model = Sequential()
model.add(
    LSTM(
        hidden_unit_count,
        batch_input_shape=(None, step_count, feature_count), # 入力 (データ数(未知数なのでNone), step数, 特徴数)
        return_sequences=False
    )
)
model.add(Dense(feature_count)) # LSTMからでた300個のノードを1つの値にまとめます。
model.add(Activation('linear')) # 活性化関数 他にもsoftmaxとかsigmoidとかreluとか
optimizer = Adam(lr=0.001) # 最適化関数にadamを利用し、学習率0.001でトレーニング

model.compile(loss='mean_squared_error', optimizer=optimizer)

In [ ]:
# jupiter上でmodelの図をみれたり画像に保存できたりするがtensorflow1.4では動かなかった。

# from IPython.display import SVG
# from tensorflow.python.keras._impl.keras.utils.vis_utils import model_to_dot 
# SVG(model_to_dot(model).create(prog='dot', format='svg'))

# from tensorflow.python.keras.utils import plot_model
# plot_model(model, to_file='model.png')

最後の出力がどうなるのか確認します。

(何個渡されるか現時点でわからないのでNone, 答え(出力)数) になります

In [ ]:
model.output_shape

### 1-3 トレーニング

早速トレーニングしてみましょう。

トレーニング中、callbackできます。

今回はcallbackに過学習(Over fitting)抑制や十分学習できた際に終了する為にEarlyStoppingを仕込みます。

学習回数(epochs)を考える手間を減らすことができます。

In [ ]:
# 30~50epochsくらいで十分学習できたと判断して終了するはず
early_stopping = EarlyStopping(
    monitor='val_loss',
    mode='auto',
    patience=20 # 値が良い方向に動かなくなった回数
)

精度を示す指標の計算方法もありますが、今回は学習しきった度合いであるlossを参考にしましょう。

低いほど良い学習が行われています。

In [ ]:
# トレーニング
model.fit(
    train_data, train_label,
    batch_size=300, # 指定個数ごとに重みが更新される
    epochs=100, # 指定回数学習
    validation_split=0.1, # 指定割合のデータをテスト用に利用する
    callbacks=[early_stopping]
)

### 1-4 予測

まずはトレーニングしたデータに対して予測してみましょう

In [ ]:
predictions = model.predict(train_data)

train_dataは75個なので75個の予測値が帰ってきます

In [ ]:
predictions.shape

データセットと予測した値を並べて正しそうか確認します

In [ ]:
plt.figure()
plt.plot(range(0, len(predictions)), predictions, color='r', label='predictions')
plt.plot(range(0, len(train_label)), train_label, color='b', label='training_data')
plt.legend()
plt.show()

これだけでは面白くないので、未来の値を予測します。

トレーニングデータの最後からstep分抜き出して予測

その予測結果も含め、最後からstep分抜き出して予測...

を繰り返して予測します。

In [ ]:
latest_data = train_data[len(train_data)-1]
results = np.empty((0, 0))
for _ in range(50):
    # 予測
    test_data = np.reshape(latest_data, (1, 25, 1))
    batch_predict = model.predict(test_data)

    # 結果蓄積
    results = np.append(results, batch_predict)

    # 次のデータをセット
    latest_data = np.delete(latest_data, 0)
    latest_data = np.append(latest_data, batch_predict)

# (50)から(50,1)に形状を変える
results = np.reshape(results, (results.shape[0], 1))

In [ ]:
plt.figure()
plt.plot(range(0, len(predictions)), predictions, color='r', label='predictions')
plt.plot(range(0, len(train_label)), train_label, color='b', label='training_data')
plt.plot(range(len(train_label)-1, len(train_label)-1+len(results)), results, color='g', label='future_predictions')
plt.legend()
plt.show()

## 2 アイスクリームの売り上げを予測 (売上だけではなく気温なども考慮した予測)

### 2-1 データセットの作成

先ほどは1つの値だけで予測していましたが、今度は売り上げや気温など、予測に影響しそうな要素も含めて学習させます。

実際のデータの方が面白いので東京の気温とアイスクリームの売り上げデータを用意し、利用しやすいように加工しました。

[参考データ](https://oku.edu.mie-u.ac.jp/~okumura/stat/160118.html)

In [ ]:
# tableのような扱いができるpandas、これまたnumpyのようなarray like objectです。
import pandas

読み込んで表示してみましょう

- year 年
- month 月
- avg_max_temperature 平均最高気温
- sum_precipitation_mm 降水量
- avg_humidity_per 平均湿度
- 25c_days 25度を超えた日数
- ice_sale アイスの売り上げ

In [ ]:
data = pandas.read_csv('volume/datasets/tokyo-weather-and-ice-sales.csv')
data

In [ ]:
plt.plot(data['avg_max_temperature'], label='avg_max_temperature') # 最大気温平均
plt.plot(data['sum_precipitation_mm'], label='sum_precipitation_mm') # 合計降水量
plt.plot(data['avg_humidity_per'], label='avg_humidity_per') # 平均湿度
plt.plot(data['25c_days'], label='25c_days') # 25度以上の日数
plt.plot(data['ice_sale'], label='ice_sale') # アイスの売り上げ(円)
plt.legend()
plt.show()

アイスの売り上げを予測する為に各データを加工します。

In [ ]:
temps = data['avg_max_temperature'].values.tolist() # 配列化
temps = np.array(temps).reshape((len(temps), 1)).astype(float) # 形状変更 (120, 1)
print(temps.shape)

precs = data['sum_precipitation_mm'].values.tolist()
precs = np.array(precs).reshape((len(precs), 1)).astype(float)
print(precs.shape)

humidities = data['avg_humidity_per'].values.tolist()
humidities = np.array(humidities).reshape((len(humidities), 1)).astype(float)
print(humidities.shape)

up25days = data['25c_days'].values.tolist()
up25days = np.array(up25days).reshape((len(up25days), 1)).astype(float)
print(up25days.shape)

icesales = data['ice_sale'].values.tolist()
icesales = np.array(icesales).reshape((len(icesales), 1)).astype(float)
print(icesales.shape)

各データの範囲がバラバラの為、学習に時間が掛かります(数値が高いデータほど重みが大きくなるなどの悪影響もあります)

なのでデータ範囲を0~1の範囲に納める標準化/正規化を行います。今回はsklearnのMinMaxScalerを使います。

(本当は最終的に出力される値も0~1になってしまうので、データごとに単純に`n/100`とか固定値で割るなどすると後で円や度などの単位に戻しやすくなりますが、今回は割愛します)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))

normalized_temps = scaler.fit_transform(temps)
normalized_precs = scaler.fit_transform(precs)
normalized_humidities = scaler.fit_transform(humidities)
normalized_up25days = scaler.fit_transform(up25days)
normalized_icesales = scaler.fit_transform(icesales)

データを一部確認してみましょう。

In [ ]:
normalized_icesales[0:10]

前回は25stepのデータにしました。

今回は一年周期で変動する月のデータなので、12の倍数が良さそうな気がします。

24とかでも良いですが、データ自体も多いわけではないので今回は12ヶ月(過去1年)をステップとしてトレーニングしてみます。

In [ ]:
temps_train_data, temps_train_label = create_reccurent_dataset(normalized_temps, step=12)
precs_train_data, precs_train_label = create_reccurent_dataset(normalized_precs, step=12)
up25days_train_data, up25days_train_label = create_reccurent_dataset(normalized_up25days, step=12)
humidities_train_data, humidities_train_label = create_reccurent_dataset(normalized_humidities, step=12)
icesales_train_data, icesales_train_label = create_reccurent_dataset(normalized_icesales, step=12)
print(temps_train_data.shape)
print(temps_train_label.shape)
print(precs_train_data.shape)
print(precs_train_label.shape)
print(up25days_train_data.shape)
print(up25days_train_label.shape)
print(humidities_train_data.shape)
print(humidities_train_label.shape)
print(icesales_train_data.shape)
print(icesales_train_label.shape)

推論する際の形状は(データ個数, ステップ数, 特徴量)です。

特徴量は(最大気温平均, 合計降水量, 平均湿度, 25度以上の日数)の5個です。

つまり (None, 12 5)を作りたいのでnumpyでデータをマージします。

出力をアイスの売り上げだけにすると、未来予測が1個後までしかできなくなるので、5個すべて答えとして用意します。

In [ ]:
train_data = np.c_[temps_train_data, precs_train_data, up25days_train_data, humidities_train_data, icesales_train_data]
train_label = np.c_[temps_train_label, precs_train_label, up25days_train_label, humidities_train_label, icesales_train_label]
print(train_data.shape)
print(train_label.shape)

### 2-2 モデル構築

1のsin波でやったことと同じ内容ですが、入力するデータが12ステップの5個の値、出力が5個の値になります。

入力 -> (None, 12, 5) 出力 -> (None, 5)

In [ ]:
# モデル構築
step_count = 12
feature_count = 5
hidden_unit_count = 120

model = Sequential()
model.add(LSTM(hidden_unit_count, batch_input_shape=(None, step_count, feature_count)))
model.add(Dense(feature_count))
model.add(Activation('linear'))
optimizer = Adam(lr=0.003)
model.compile(loss='mean_squared_error', optimizer=optimizer)
early_stopping = EarlyStopping(monitor='val_loss', mode='auto', patience=20)

### 2-3 トレーニング

トレーニングしてみましょう。

In [ ]:
model.fit(
    train_data, train_label,
    batch_size=300,
    epochs=200,
    validation_split=0.1,
    callbacks=[early_stopping]
)

### 2-4 予測

トレーニングデータと予測結果を合わせて確認してみましょう。

多くてグラフが見辛いため特徴ごとに予測できているか確認します。

薄い色が正解(トレーニングデータ)、濃い色が予測結果です。

In [ ]:
predictions = model.predict(train_data)

In [ ]:
# 最大気温平均
plt.figure()
plt.plot(range(0, len(train_label)), train_label[:,0], color=(1.0, 0.7, 0.7), label='train_label temp')
plt.plot(range(0, len(predictions)), predictions[:,0], color=(1.0, 0.0, 0.0), label='predictions temp')
plt.legend()
plt.show()

In [ ]:
# 合計降水量
plt.figure()
plt.plot(range(0, len(train_label)), train_label[:,1], color=(0.7, 0.7, 1.0), label='train_label prec')
plt.plot(range(0, len(predictions)), predictions[:,1], color=(0.0, 0.0, 1.0), label='predictions prec')
plt.legend()
plt.show()

In [ ]:
# 平均湿度
plt.figure()
plt.plot(range(0, len(train_label)), train_label[:,2], color=(0.7, 1.0, 1.0), label='train_label humidity')
plt.plot(range(0, len(predictions)), predictions[:,2], color=(0.0, 1.0, 1.0), label='predictions humidity')
plt.legend()
plt.show()

In [ ]:
# 25度以上の日数
plt.figure()
plt.plot(range(0, len(train_label)), train_label[:,3], color=(1.0, 1.0, 0.7), label='train_label 25c_days')
plt.plot(range(0, len(predictions)), predictions[:,3], color=(1.0, 1.0, 0.0), label='predictions 25c_days')
plt.legend()
plt.show()

In [ ]:
# アイスの売り上げ
plt.figure()
plt.plot(range(0, len(train_label)), train_label[:,4], color=(0.7, 1.0, 1.0), label='train_label ice_sale')
plt.plot(range(0, len(predictions)), predictions[:,4], color=(0.0, 1.0, 1.0), label='predictions ice_sale')
plt.legend()
plt.show()

In [ ]:
# すべて(見辛い！)
plt.figure()
plt.plot(range(0, len(train_label)), train_label[:,0], color=(1.0, 0.7, 0.7), label='train_label temp')
plt.plot(range(0, len(predictions)), predictions[:,0], color=(1.0, 0.0, 0.0), label='predictions temp')
plt.plot(range(0, len(train_label)), train_label[:,1], color=(0.7, 0.7, 1.0), label='train_label prec')
plt.plot(range(0, len(predictions)), predictions[:,1], color=(0.0, 0.0, 1.0), label='predictions prec')
plt.plot(range(0, len(train_label)), train_label[:,2], color=(0.7, 1.0, 1.0), label='train_label humidity')
plt.plot(range(0, len(predictions)), predictions[:,2], color=(0.0, 1.0, 1.0), label='predictions humidity')
plt.plot(range(0, len(train_label)), train_label[:,3], color=(1.0, 1.0, 0.7), label='train_label 25c_days')
plt.plot(range(0, len(predictions)), predictions[:,3], color=(1.0, 1.0, 0.0), label='predictions 25c_days')
plt.plot(range(0, len(train_label)), train_label[:,4], color=(0.7, 1.0, 1.0), label='train_label ice_sale')
plt.plot(range(0, len(predictions)), predictions[:,4], color=(0.0, 1.0, 1.0), label='predictions ice_sale')
# plt.legend()
plt.show()

5ヶ月後の予測がどうなるかも調べてみましょう

In [ ]:
latest_data = np.array([train_data[len(train_data)-1]])
results = []
for _ in range(5):
    # 推論
    print(latest_data.shape)
    batch_predict = model.predict(latest_data)
    # 結果蓄積
    results.append([batch_predict[0][0], batch_predict[0][1], batch_predict[0][2], batch_predict[0][3], batch_predict[0][4]])
    # 次のデータをセット
    latest_data = np.delete(latest_data, np.array([batch_predict]), axis=1)
    latest_data = np.append(latest_data, np.array([batch_predict]), axis=1)
results = np.array(results)

In [ ]:
plt.figure()

plt.plot(range(0, len(train_label)), train_label[:,0], color=(1.0, 0.7, 0.7), label='train_label temp')
plt.plot(range(0, len(train_label)), train_label[:,1], color=(0.7, 0.7, 1.0), label='train_label prec')
plt.plot(range(0, len(train_label)), train_label[:,2], color=(0.7, 1.0, 1.0), label='train_label humidity ')
plt.plot(range(0, len(train_label)), train_label[:,3], color=(1.0, 1.0, 0.7), label='train_label 25c_days')
plt.plot(range(0, len(train_label)), train_label[:,4], color=(0.7, 1.0, 1.0), label='train_label ice_sale')

plt.plot(range(len(train_label)-1, len(train_label)-1+len(results)), results[:,0], color=(1.0, 0.0, 0.0), label='future temp')
plt.plot(range(len(train_label)-1, len(train_label)-1+len(results)), results[:,1], color=(0.0, 0.0, 1.0), label='future prec')
plt.plot(range(len(train_label)-1, len(train_label)-1+len(results)), results[:,2], color=(0.0, 1.0, 1.0), label='future humidity')
plt.plot(range(len(train_label)-1, len(train_label)-1+len(results)), results[:,3], color=(1.0, 1.0, 0.0), label='future 25c_days')
plt.plot(range(len(train_label)-1, len(train_label)-1+len(results)), results[:,4], color=(0.0, 1.0, 1.0), label='future ice_sales')

# plt.legend()
plt.show()

## 3 規則性のないbitcoinの値段を予測 (最終取引価格)

規則性のあるデータは分析すれば機械学習を使わなくてもある程度予測は可能ですね。

今度は規則性のない(見つけづらい)bitcoinの値段を使って学習してみましょう。

### 3-1 データセットの作成

データセットにbitflyerさんのpybitflyerを利用させていただきます。

pybitflyerはアカウントいらずでデータを収集できます。

In [ ]:
import pybitflyer

対象のデータはbitcoinの日本円価格とします

In [ ]:
api = pybitflyer.API()
data = api.ticker(product_code='BTC_JPY')

In [ ]:
data

様々なデータがありますが、この辺りが利用できそうですね。

In [ ]:
print(data['ltp']) # 最終取引価格
print(data['best_ask']) # 最高買い価格
print(data['best_bid']) # 最小売り価格

このデータを用途に応じて数分おき、数時間おきに取得します。

5秒分のデータをとって見ましょう。

In [ ]:
import time
from datetime import datetime

In [ ]:
btc_jpy_data = []
count = 5
api = pybitflyer.API()
while True:
    tick = api.ticker(product_code='BTC_JPY')
    print('tick={} ltp={}'.format(len(btc_jpy_data), tick['ltp']))
    btc_jpy_data.append(tick)
    time.sleep(1)
    if count <= len(btc_jpy_data):
        break

In [ ]:
btc_jpy_data

迷惑のかからない適度な利用にとどめたいですね。

1000秒分のデータをjsonで保存してあるのでそれを利用します。

In [ ]:
# ファイルから
import json
with open('volume/datasets/BTC_JPY.json', 'r') as f:
    btc_jpy_data = json.load(f)

In [ ]:
btc_jpy_data

最終取引価格、最高買い価格、最低売り価格を利用してデータセットを作ります

In [ ]:
ltp_data = np.array([[d['ltp']] for d in btc_jpy_data]) # 最終取引価格
best_ask_data = np.array([[d['best_ask']] for d in btc_jpy_data]) # 最高買い価格
best_bid_data = np.array([[d['best_bid']] for d in btc_jpy_data]) # 最小売り価格

以前同様、標準化/正規化します (すべての値を0~1の範囲に納める)

In [ ]:
# 標準化
scaler = MinMaxScaler(feature_range=(0, 1))
ltp_data = scaler.fit_transform(ltp_data)
best_ask_data = scaler.fit_transform(best_ask_data)
best_bid_data = scaler.fit_transform(best_bid_data)

データの内容をみてみましょう

In [ ]:
plt.figure()
plt.plot(range(0, len(ltp_data)), ltp_data, color='r', label='ltp')
plt.plot(range(0, len(best_ask_data)), best_ask_data, color='g', label='best_ask')
plt.plot(range(0, len(best_bid_data)), best_bid_data, color='b', label='best_bid')
plt.legend()
plt.show()

5step(過去5秒を元に1秒後を予測)でデータを整えてみます。

step数は自由に変えてもらって構いません。

In [ ]:
ltp_train_data, ltp_train_label = create_reccurent_dataset(ltp_data, step=5)
best_ask_train_data, best_ask_train_label = create_reccurent_dataset(best_ask_data, step=5)
best_bid_train_data, best_bid_train_label = create_reccurent_dataset(best_bid_data, step=5)

In [ ]:
print(ltp_train_data.shape)
print(best_ask_train_data.shape)
print(best_bid_train_data.shape)

In [ ]:
print(ltp_train_label.shape)
print(best_ask_train_label.shape)
print(best_bid_train_label.shape)

推論する際の形状 = (個数, ステップ数, 特徴量)

今回の推論時の形状は入力も出力もltp, best_ask, best_bidの3つなので`(個数, 5, 3)`になります

In [ ]:
train_data = np.c_[ltp_train_data, best_ask_train_data, best_bid_train_data]
train_label = np.c_[ltp_train_label, best_ask_train_label, best_bid_train_label]
print(train_data.shape)
print(train_label.shape)

### 3-2 モデル構築

モデルを作成します。

不安定な場合は学習率を上げ下げするといいでしょう。`Adam(lr=0.0001)`

パラメータの調整はグリッドサーチなどパラメータ調整は色々ありますが、

今回は使わないので低い学習率から3倍ずつ上げる方法が推奨されていました。 0.001 -> 0.003 -> 0.03 -> 0.01

In [ ]:
# モデル構築
step_count = 5
feature_count = 3
hidden_unit_count = 300

model = Sequential()
model.add(LSTM(hidden_unit_count, batch_input_shape=(None, step_count, feature_count), return_sequences=False))
model.add(Dense(feature_count))
model.add(Activation('linear'))
optimizer = Adam(lr=0.001)
model.compile(loss='mean_squared_error', optimizer=optimizer)
early_stopping = EarlyStopping(monitor='val_loss', mode='auto', patience=20)

### 3-3 トレーニング

トレーニング実行してみましょう。

In [ ]:
# トレーニング
model.fit(
    train_data, train_label,
    batch_size=2000,
    epochs=200,
    validation_split=0.1,
    callbacks=[early_stopping]
)

## 3-4 予測

トレーニングデータと予測結果を検証してみましょう。

In [ ]:
predictions = model.predict(train_data)

In [ ]:
plt.figure()
plt.plot(range(0, len(train_label)), train_label[:,0], color=(1.0, 0.7, 0.7), label='train_label ltp')
plt.plot(range(0, len(train_label)), train_label[:,1], color=(0.7, 1.0, 0.7), label='train_label best_ask')
plt.plot(range(0, len(train_label)), train_label[:,2], color=(0.7, 0.7, 1.0), label='train_label best_bid')
plt.plot(range(0, len(predictions)), predictions[:,0], color=(1.0, 0.0, 0.0), label='predictions ltp')
plt.plot(range(0, len(predictions)), predictions[:,1], color=(0.0, 1.0, 0.0), label='predictions best_ask')
plt.plot(range(0, len(predictions)), predictions[:,2], color=(0.0, 0.0, 1.0), label='predictions best_bid')
plt.legend()
plt.show()

50秒後まで予測してみます

In [ ]:
latest_data = np.array([train_data[len(train_data)-1]])
results = []
for _ in range(50):
    # 推論
    batch_predict = model.predict(latest_data)
    # 結果蓄積
    results.append([batch_predict[0][0], batch_predict[0][1], batch_predict[0][2]])
    # 次のデータをセット
    latest_data = np.delete(latest_data, np.array([batch_predict]), axis=1)
    latest_data = np.append(latest_data, np.array([batch_predict]), axis=1)
results = np.array(results)
results.shape

In [ ]:
plt.figure()
plt.plot(range(0, len(train_label)), train_label[:,0], color=(1.0, 0.7, 0.7), label='train_label ltp')
plt.plot(range(0, len(train_label)), train_label[:,1], color=(0.7, 1.0, 0.7), label='train_label best_ask')
plt.plot(range(0, len(train_label)), train_label[:,2], color=(0.7, 0.7, 1.0), label='train_label best_bid')
plt.plot(range(len(train_label)-1, len(train_label)-1+len(results)), results[:,0], color=(1.0, 0.0, 0.0), label='future ltp')
plt.plot(range(len(train_label)-1, len(train_label)-1+len(results)), results[:,1], color=(0.0, 1.0, 0.0), label='future best_ask')
plt.plot(range(len(train_label)-1, len(train_label)-1+len(results)), results[:,2], color=(0.0, 0.0, 1.0), label='future best_bid')
plt.legend()
plt.show()

トレーニング状況にもよりますが、試した際にはすこし上昇する予測がでました。

## まとめ

- リアルタイムにデータをとって学習させ、数日放置するのも良さそうです。
- 単位がわかりにくいので特徴別に標準化する固定値を入れても良いかも
- 特徴量を増やすと良いかもしれない 最大、最小、ローソク足の最大最小、関連ニュース数(あわよくば良し悪し別)、曜日
- 予測できないイベント(プレスリリースが出た、ハッキングによって流出、政府による規制が発表されたなど)によって上下するので、後に出てくるデータではその不測は予測できないだろう
- あくまでも傾向として参考程度にはなりそう

## おまけ word2vec


単語を入力として、その周辺にどのような単語が現れやすいか予測するニューラルネットワーク(Skip-gram)があります。

そのネットワークの中間層(隠れ層)の重みを単語ベクトルとして活用しよう！というのがword2vecです。

単語ベクトルは座標です。 e.g. [0.74, 0.29, 0.98, 0.01, 0.13, 0.35]

地図は緯度経度の2つが座標、高度が加わると3つの座標で高さもわかる、時間軸追加して...

それ以降は人間にはイメージしにくいですが、その座標に近い単語だったり、座標の足し算引き算を行って単語の四則演算ができるようになります。

[参考](https://deepage.net/bigdata/machine_learning/2016/09/02/word2vec_power_of_word_vector.html)

まずはためしてみましょう！

### データセット

まずは元となるテキストを用意します。

[青空文庫](https://www.aozora.gr.jp/cards/001847/card57347.html)さん等からテキストデータをダウンロードしてきましょう。

slackの会話データとってきたりしても面白そう。

テキストの中を確認して必要な文だけに絞ってください。

`volume/datasets`などに配置して、適宜pathを書き換えてください。

corrected_text_pathは加工したテキストを保存するpathです。

In [ ]:
text_path = 'volume/datasets/book.txt'
corrected_text_path = 'volume/datasets/book_corrected.txt'

データはmecabで形態素解析してわかちがきした(基本形に統一した)テキストを用意します。

具体的には下記のようになります

'彼は老いていた' ->'彼,は,老いる,て,いる,た '

Dockerfileにinstall mecab-ipadicと書かれてますが、新しい日本語辞書(と言っても2016-05-02ですが)を含めて、最近の言葉に対応できるようになっています。

サービス独自の単語がある場合は辞書を作ってみるのもいいでしょう。

In [ ]:
import codecs
import MeCab

適宜使用するテキストの文字コードに応じて書き換えましょう

In [ ]:
with codecs.open(text_path, 'r', 'shift-jis') as file:
    texts = file.read()
lines = texts.split("\r\n")

In [ ]:
lines

空白やハイフンを除去して整形したテキストファイルを作ります

In [ ]:
tagger = MeCab.Tagger('-Owakati')

with codecs.open(corrected_text_path, 'w') as corrected_file:
    for line in lines:
        if not line:
            # 空行を除く処理
            continue
        if line[0] == "-":
            # ハイフンの処理
            continue
        corrected_file.write(tagger.parse(line))


with codecs.open(corrected_text_path, 'r') as corrected_file:
    doc = corrected_file.read()
doc

整形したテキストをword2vecで学習します。

In [ ]:
from gensim.models import word2vec
sentences = word2vec.LineSentence(corrected_text_path)

model = word2vec.Word2Vec(
    sentences, 
    sg=1, # 1=skip-gram, 0=C-BOW
    size=100, # 単語の次元数
    min_count=1, # n回未満登場する単語を破棄 頻出のみ学習したいのなら上げる
    window=10, # 周囲10個の単語に対して
    hs=1, # 階層化ソフトマックス使用フラグ
    negative=0 # ネガティブサンプリングに用いる単語数 ドロップアウトのようなもの
)

類似語と類似度をしらべるにはmost_similarメソッドを利用します

In [ ]:
model.wv.most_similar(positive=['人'])

テキストが少ないのでそこまで良い結果にはなってないはずです。

もっと大量にテキストデータとトレーニングする時間が必要なのですが、wikiのテキストを学習したモデルが公開されていましたのでそれを利用させていただきます！

[word2vecの学習済み日本語モデルを公開します 白ヤギコーポレーション](http://aial.shiroyagi.co.jp/2017/02/japanese-word2vec-model-builder/)

In [ ]:
from gensim.models.word2vec import Word2Vec

In [ ]:
model = Word2Vec.load('volume/models/latest-ja-word2vec-gensim-model/word2vec.gensim.model')

単語ベクトルの数とその数値をみてみましょう

In [ ]:
model.wv['ピザ'].shape

In [ ]:
model.wv['ピザ']

類似語と類似度をしらべるにはmost_similarメソッドを利用します

positiveが近い単語、negativeが遠い単語です。

In [ ]:
model.wv.most_similar(positive=['ピザ']) # or model.similar_by_word('ピザ')

In [ ]:
model.wv.most_similar(negative=['ピザ'])

単語と単語間の類似度、距離を調べるにはsimilarityメソッドを使います

In [ ]:
# 類似度
print(model.wv.similarity('ピザ', 'ハンバーガー'))
print(model.wv.similarity('ピザ', 'パン'))
print(model.wv.similarity('ピザ', 'スプーン'))
print(model.wv.similarity('ピザ', 'イス'))

similar_by_vectorメソッドで、単語ではなくベクトル値から近い単語を出力できるので、

In [ ]:
# ベクトルから類似語と類似度
vector = model.wv['ピザ']
model.wv.similar_by_vector(vector)

ベクトル値を計算した後の近い単語を出力できます。

In [ ]:
vector = model.wv['王様'] - model.wv['じいさん'] + model.wv['女']
model.wv.similar_by_vector(vector)

In [ ]:
vector = model.wv['イチロー'] - model.wv['野球'] + model.wv['サッカー']
model.wv.similar_by_vector(vector)

In [ ]:
vector = model.wv['宇宙人'] - model.wv['人']
model.wv.similar_by_vector(vector)

### 何に利用できるか

- 単語から類似度が取れるので関連商品を表示するなどレコメンと
- レコメンドと似たようなものだが、関連性の高い質問と紐づけるQ&Aへの利用
- 他にも機械翻訳や感情分析にも利用されている事例がある

### 弱点

単語の周囲の文字から単語ベクトルを学習するので、対義語を出すことができません。

具体的には、『あなた が 好き です』『あなた が 嫌い です』のような文にたいして、

`が`の次が`好き`や`嫌い`だと学習し、

`です`の前が`好き`や`嫌い`だと学習するため、

`好き`と`嫌い`のベクトルは近くなってしまいます。

実際 `ピザ`からかけ離れたベクトルは`CNT`や`基き`になり、全くねじれた方向の単語になります。

ただ、`インストール`に近いベクトルに`アンインストール`などの対義語が含まれるので、

『インストール方法』を検索した人に対して『アンインストール方法』をレコメンドするなど、この特性はレコメンドとしては良い弱点になりそうな気がします。